Testing FNO

In [1]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

In [2]:
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[2]
github_root = Path.cwd().resolve().parents[1]
sys.path.append(str(project_root))
sys.path.append(str(github_root))

print(project_root)
print(github_root)

C:\SenkDosya\Projects\AeroML
C:\SenkDosya\Projects\AeroML\2D_FlowField


In [3]:
from src.models.FNO.FNO import *
from src.utils.metrics import get_n_params

# Use cuda

device = torch.device('cuda')

In [4]:


cfg_fno = {
    'cfg_p': {
       'in_channels': 1,
        'out_channels': 64,
        'hidden_channels': [16,8],
        'dropout': 0.05,
        'act_fn': 'gelu' 
    },
    'cfg_q': {
        'in_channels': 64,
        'out_channels': 1,
        'hidden_channels': [8,16],
        'dropout': 0.05,
        'act_fn': 'gelu'
    },
    'fb_hidden_channels': [32,32],
    'fb_modes': [[25,25],[25,25],[25,25]],
    'fb_kernel': [1,1,1],
    'fb_factorization': None,
}



Fno = FNO_nD(**cfg_fno)

Fno = Fno.to(device)

print(get_n_params(Fno))
        

3235825


In [5]:
batch_size = 10
seq_length = 225
input_dim = 1


x = torch.randn(size=(batch_size, input_dim, seq_length, seq_length)).to(device)





In [6]:
y_pred = Fno(x)

In [7]:

loss = nn.MSELoss()

pred_loss = loss(y_pred, y_act)




NameError: name 'y_act' is not defined

In [ ]:
# Get gradients
pred_loss.backward()

In [ ]:
x = torch.randn(size=(batch_size, input_dim, seq_length+10, seq_length+10)).to(device)

y_pred = Fno(x)

In [ ]:
import time

# Setup
torch.backends.cudnn.benchmark = True
Fno.eval()

# Dummy Data (Use 256 for Power-of-2 speedup test)
# If this is faster than your 225 run, implement padding!
x_test = torch.randn(10, 1, 256, 256).to(device)

# Warmup
with torch.inference_mode(), torch.autocast(device_type='cuda', dtype=torch.float32):
    for _ in range(5): _ = Fno(x_test)

# Benchmark
torch.cuda.synchronize()
t0 = time.time()

with torch.inference_mode(), torch.autocast(device_type='cuda', dtype=torch.float32):
    for _ in range(20):
        _ = Fno(x_test)

torch.cuda.synchronize()
print(f"Optimized Time: {(time.time() - t0) / 20:.5f} s")

Optimized Time: 1.29753 s


In [ ]:
# 1. Setup Static Input (Must be exact shape used for capture)
static_input = torch.randn(10, 1, 225, 225, device=device) # Batch 10
static_output = torch.empty(10, 1, 225, 225, device=device) # Pre-allocate output

# 2. Warmup (Run normal pytorch a few times)
s = torch.cuda.Stream()
s.wait_stream(torch.cuda.current_stream())
with torch.cuda.stream(s):
    for _ in range(3):
        _ = Fno(static_input)
torch.cuda.current_stream().wait_stream(s)

# 3. Capture
g = torch.cuda.CUDAGraph()
# Sets up the recording
with torch.cuda.graph(g):
    static_output = Fno(static_input)

# 4. Replay (This is the fast part)
def run_fast_inference(new_data):
    # Copy new data into the "static" memory slot
    static_input.copy_(new_data)
    # Replay the recorded graph
    g.replay()
    # Return result (clone if you need to keep it, otherwise it gets overwritten next run)
    return static_output.clone()

# --- BENCHMARK IT ---
import time
torch.cuda.synchronize()
t0 = time.time()

for _ in range(20):
    y_pred = run_fast_inference(x) # x is your actual data

torch.cuda.synchronize()
print(f"CUDA Graph Time: {(time.time() - t0) / 20:.5f} s")

CUDA Graph Time: 0.18929 s
